In [1]:
using TensorKit
import TensorKit.×
using KrylovKit
include("./iPEPS/iPEPS.jl")
include("./iPEPS/swap_gate.jl")
include("./iPEPS/util.jl")

inv

In [ ]:
## follow convention (l, t, σ, r, b)
## domain: r, b
## codomain: l, t, σ
M11 = TensorMap(randn, ComplexF64, ℂ^3 ⊗ ℂ^3 ⊗ ℂ^2, ℂ^3 ⊗ ℂ^3)
@show domain(M11)
@show space(M11)
M12 = TensorMap(randn, ComplexF64, ℂ^3 ⊗ ℂ^3 ⊗ ℂ^2, ℂ^3 ⊗ ℂ^3)

## contraction 后用; 分割 (codomain; domain), 原来的domain 放到 codomain，或反过来，空间都要变为对偶空间（加'）
@tensor M1112[l1, t1, σ1, σ2, b1; t2, r2, b2] := M11[l1, t1, σ1, r1, b1] * M12[r1, t2, σ2, r2, b2]
# B = permute(A, (3,), (1, 2, 4, 5))
# @show domain(B)
@show domain(M1112)
@show codomain(M1112)

In [5]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 1, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((1, 1//2) => 1, (0, 1) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet
# @show typeof(pspace)
# @show GradedSpace isa ElementarySpace
# @show pspace isa VectorSpace
# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)

Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)

In [5]:
A = id(pspace)
@show A
exp!(A)
@show A
nothing

A = TensorMap(Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)):
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()):
 1.0  0.0
 0.0  1.0
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()):
 1.0

A = TensorMap(Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)):
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()):
 2.718281828459045  0.0
 0.0                2.718281828459045
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()):
 2.718281828459045



In [6]:
include("./simple_update/iPEPS_GammaLambda.jl")
ipeps = iPEPSΓΛ(pspace, aspacel, aspacet, 2, 2; dtype=ComplexF64)

gateNN = TensorMap(randn, ComplexF64, pspace ⊗ pspace, pspace ⊗ pspace)

nothing

In [25]:
xx = 1
yy = 1
Dk = 5
Γl = ipeps[xx, yy].Γ
Γr = ipeps[xx+1, yy].Γ
swgate = swap_gate(space(Γl)[3]', space(Γl)[5]')
@tensor Γl[l, t, p; r, b] = Γl[le, te, pin, r, be] * ipeps[xx, yy].l[l, le] * ipeps[xx, yy].t[t, te] *
                            ipeps[xx, yy].b[be, bin] * swgate[pin, bin, p, b]
@tensor Γr[l, t, p; r, b] = Γr[l, te, p, re, be] * ipeps[xx+1, yy].t[t, te] * ipeps[xx+1, yy].r[re, r] * ipeps[xx+1, yy].b[be, b]
Xl, vl = leftorth(Γl, ((1, 2, 5), (3, 4)))
wr, Yr = rightorth(Γr, ((1, 3), (2, 4, 5)))
@tensor mid[toX, pl, pr; toY] := vl[toX, plin, toV] * ipeps[xx, yy].r[toV, toW] * gateNN[pl, pr, plin, prin] *
                                 wr[toW, prin, toY]
vnew, λnew, wnew, err = tsvd(mid, ((1, 2), (3, 4)); trunc=truncdim(Dk))
ipeps[xx, yy].r = λnew
ipeps[xx+1, yy].l = λnew
swgate = swap_gate(space(vl)[2]', space(Xl)[3]')
@tensor Γl[l, t, p; r, b] = Xl[] * vnew[] * swgate[] * 
                        inv(ipeps[xx, yy].l)[] * inv(ipeps[xx, yy].t)[] * inv(ipeps[xx, yy].b)[]
nothing

space(mid) = (Rep[ℤ₂ × SU₂]((0, 0)=>3, (1, 1/2)=>3, (0, 1)=>1) ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)) ← Rep[ℤ₂ × SU₂]((0, 0)=>3, (1, 1/2)=>3, (0, 1)=>1)
space(λnew) = Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>2) ← Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>2)
space((ipeps[xx, yy]).r) = Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)


In [30]:
swgate = swap_gate(space(ipepsΓΛ[1,1].Γ)[3]', space(ipepsΓΛ[1,1].Γ)[5]')
@tensor ipepsΓΛ[1,1].Γ[l, t, p; r, b] = ipepsΓΛ[1,1].Γ[l, t, pin, r, bin] * swgate[pin, bin, p, b]
@show space(ipepsΓΛ[1,1].Γ)
nothing

space((ipepsΓΛ[1, 1]).Γ) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1) ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)) ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1))
